In [1]:
import hickle as hkl
import numpy as np

from lib.read_data import u_desvio as u_true
from lib.read_data import y_desvio as y_true

ny = y_true.shape[1]
nu = u_true.shape[1]
theta_arx = hkl.load("../export/ARX.hkl")

A = theta_arx[:ny, :].T
B = theta_arx[ny:, :].T
C = np.eye(ny)
D = np.zeros((ny, nu * 2))


def model_true(x: np.ndarray, u_ext: np.ndarray):
    if x.shape != (ny,):
        raise ValueError(f"x inválido: shape {x.shape}, esperado {(ny,)}")

    if u_ext.shape != (2 * nu,):
        raise ValueError(f"u_ext inválido: shape {u_ext.shape}, esperado {(2 * nu,)}")

    x = A @ x + B @ u_ext
    y = C @ x + D @ u_ext
    return y


def model_linear(x: np.ndarray, u_ext: np.ndarray):
    if x.shape != (ny,):
        raise ValueError(f"x inválido: shape {x.shape}, esperado {(ny,)}")

    if u_ext.shape != (2 * nu,):
        raise ValueError(f"u_ext inválido: shape {u_ext.shape}, esperado {(2 * nu,)}")

    x = A @ x + B @ u_ext
    y = C @ x + D @ u_ext
    return y


In [2]:
def y_to_x(y):
    x = y  # Sistema diretamente observável
    return x


In [3]:
from scipy.optimize import minimize

Np = 10  # horizonte de predição
Nc = 3  # horizonte de controle
Q = np.full(ny, 1.0)  # peso no erro
R = np.full(nu, 0.5)  # peso na variação da entrada


def J(u_seq, x0, sp, u0):
    """
    u_seq: sequência de entradas (Nc,)
    x0: estado atual
    sp: setpoint
    u0: última entrada aplicada
    """
    u_seq = u_seq.reshape((Nc, nu))
    x = x0
    cost = 0.0

    u_prev = u0
    for k in range(Np):
        # Usa a última entrada se k >= Nc
        if k < Nc:
            u = u_seq[k]
        else:
            u = u_seq[-1]

        u_ext = np.concatenate([u_prev, u])

        y = model_linear(x, u_ext)
        e = y - sp  # shape (ny,)
        cost += e.T @ np.diag(Q) @ e
        x = y_to_x(y)

        u_prev = u

    u_old = u0
    for k in range(Nc):
        u = u_seq[k]

        du = u - u_old
        cost += du.T @ np.diag(R) @ du

        u_old = u

    return cost


def MPC(x0, sp, u_old):
    """
    x0: estado atual
    sp: setpoint
    u_old: última entrada aplicada
    """
    u0 = np.tile(u_old, Nc).flatten()  # chute inicial: repete u_old Nc vezes

    # Otimização
    res = minimize(J, u0, args=(x0, sp, u_old), method="SLSQP")

    # Retorna a primeira entrada da sequência otimizada
    return np.array(res.x).reshape((Nc, nu))[0]


# Simulando malha fechada


In [4]:
ncycles = 100
cycles = np.arange(0, ncycles)

# Prealocando histórico de saída e entrada
y = np.zeros((ncycles, ny))
y[0] = y_true[0]
u = np.zeros((ncycles, nu))
u[0] = u_true[0]

sp = y_true[-1]  # setpoint

x = y_to_x(y[0])
for k in range(1, ncycles):
    u[k] = MPC(x, sp, u[k - 1])

    # Atualiza o estado
    y[k] = model_true(y[k - 1], np.concatenate([u[k - 1], u[k]]))
    x = y_to_x(y[k])


In [5]:
from lib.read_data import u_ref, y_ref

y = y + np.tile(y_ref, (y.shape[0], 1))
sp = sp + y_ref

u = u + np.tile(u_ref, (u.shape[0], 1))

print(sp.shape)


(5,)


In [6]:
# Plotando gráficos
from lib.plot import colors, legend_outside, plt

# --- Configuração das variáveis ---
y_labels = ["Purity H2", "H2/CO ratio", "Purity CO2", "Recovery CO2", "Productivity"]
y_data = [y[:, i] for i in range(5)]
sp_data = [sp[i] for i in range(5)]

u_labels = ["$t_{feed}$", "$t_{rinse}$", "$t_{blow}$", "$t_{purge}$"]
u_data = [u[:, i] for i in range(4)]

group1 = [0, 2, 3]  # purity H2, purity CO2, recovery CO2
group2 = [1, 4]  # ratio, productivity

# --- Criar subplots ---
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(10, 12))

# Gráfico 1 – Entradas
for label, series in zip(u_labels, u_data):
    ax1.plot(cycles, series, label=label)

ax1.set_ylabel("Duração da etapa (s)")
ax1.grid(True, linestyle=":", alpha=0.4)
legend_outside(ax1)

# Gráfico 2 – Group 1
for i in group1:
    ax2.plot(cycles, y_data[i], label=y_labels[i], color=colors[i])
    ax2.plot(cycles, np.full_like(cycles, sp_data[i]), "--", color=colors[i])

ax2.set_ylabel("Indicadores (Grupo 1)")
ax2.grid(True, linestyle=":", alpha=0.4)
legend_outside(ax2)

# Gráfico 3 – Group 2
for i in group2:
    ax3.plot(cycles, y_data[i], label=y_labels[i], color=colors[i])
    ax3.plot(cycles, np.full_like(cycles, sp_data[i]), "--", color=colors[i])

ax3.set_xlabel("Ciclo")
ax3.set_ylabel("Indicadores (Grupo 2)")
ax3.grid(True, linestyle=":", alpha=0.4)
legend_outside(ax3)

plt.tight_layout()
plt.savefig("../figures/MPC.png")
plt.close()
